In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=2b51968438416fe14aaea047024c2ace295f66e28c5f49b00b0748c0579b7b79
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [6]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import torch
from torch.utils.data import DataLoader, TensorDataset

sns.set_theme()
tqdm.pandas()

# Modular function for the end-to-end combined pipeline:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
project_path = "/content/drive/MyDrive/University/Winter 23 Courses/CSI 5180 - AI Virtual Assistants/Project"

kmeans_model_path = f"{project_path}/models/combined_approach/kmeans.pkl"
bert_model_path = f"{project_path}/models/bert_inscope_original_data/saved_models"

In [7]:
kmeans = pickle.load(open(kmeans_model_path, 'rb'))
mpnet = SentenceTransformer("all-mpnet-base-v2")

tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_path)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
bert_model.to(device)
bert_model.eval()

THRESHOLD = 0.95
df_test = pd.read_csv(f"{project_path}/data/original_dataset/original_test_data.csv", dtype={"query":"string","label":"string"})
class_labels = sorted(df_test["label"].unique())
class_labels.remove('oos')

In [31]:
def compute_distance_of_closest_cluster(df):
    cluster_distances = kmeans.transform(mpnet.encode(df["query"].values))
    cluster_min_distances = cluster_distances.min(axis=1)
    df["min_distances"] = cluster_min_distances

    return df

def predict_oos(df):
    '''
    input df is a Pandas dataframe with the text input under the column named 'query'
    ''' 
    sentence_embeddings = []
    embedding_matrix = mpnet.encode(df["query"].values)
    for i in range(embedding_matrix.shape[0]):
        sentence_embeddings.append(embedding_matrix[i])  

    df["mpnet_embeddings_sentence"] = sentence_embeddings
    df = compute_distance_of_closest_cluster(df)
    df["pred_1"] = df.min_distances.apply(lambda x: "oos" if x>THRESHOLD else "in-scope")

    return df

def predict_inscope(df):
    query_list = df_test["query"].to_list()
    encoded_query_list = tokenizer(query_list, padding=True, truncation=True, return_tensors='pt').to(device)
    input_ids = encoded_query_list['input_ids']
    token_type_ids = encoded_query_list['token_type_ids']
    attention_mask = encoded_query_list['attention_mask']

    input_data = TensorDataset(input_ids, token_type_ids, attention_mask)
    dataloader = DataLoader(input_data, batch_size=64)

    predictions = []
    for batch_input_ids, batch_token_type_ids, batch_attention_mask in dataloader:
        with torch.no_grad():
            logits = bert_model(input_ids=batch_input_ids, token_type_ids=batch_token_type_ids, attention_mask=batch_attention_mask)[0]
            batch_predictions = torch.argmax(logits, dim=1)

        predictions.extend(batch_predictions.tolist())

    df['pred_2'] = ''
    for i, row in df.iterrows():
        if row['pred_1'] == 'oos':
            df.at[i, 'pred_2'] = 'oos'
        else:
            df.at[i, 'pred_2'] = class_labels[predictions[i]]

    return df

def predict_intent(df):

    df = predict_oos(df)
    df = predict_inscope(df)

    return df


In [32]:
df_results = predict_intent(df_test)

In [37]:
report = classification_report(df_results["label"], df_results["pred_2"])

In [39]:
print(report)

                           precision    recall  f1-score   support

      accept_reservations       0.94      0.97      0.95        30
          account_blocked       0.90      0.93      0.92        30
                    alarm       1.00      0.97      0.98        30
       application_status       1.00      1.00      1.00        30
                      apr       0.94      1.00      0.97        30
            are_you_a_bot       0.94      1.00      0.97        30
                  balance       0.91      1.00      0.95        30
             bill_balance       0.90      0.87      0.88        30
                 bill_due       0.81      0.97      0.88        30
              book_flight       0.91      0.97      0.94        30
               book_hotel       0.94      1.00      0.97        30
               calculator       0.85      0.93      0.89        30
                 calendar       0.93      0.90      0.92        30
          calendar_update       0.97      0.97      0.97     

In [18]:
df_test["binary_label"] =  df_test.label.apply(lambda x: "oos" if x=="oos" else "in-scope")
pd.DataFrame(classification_report(df_test["binary_label"], df_test["pred"], output_dict=True)).T

,precision,recall,f1-score,support
in-scope,0.950548,0.982444,0.966233,4500.000000
oos,0.906949,0.770000,0.832883,1000.000000
accuracy,0.943818,0.943818,0.943818,0.943818
macro avg,0.928749,0.876222,0.899558,5500.000000
weighted avg,0.942621,0.943818,0.941988,5500.000000
